In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar 22 11:49:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train_raw.csv to train_raw.csv


In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [6]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import  AutoModelForSequenceClassification, AutoTokenizer
from transformers import T5ForSequenceClassification


In [9]:
# Load T5 tokenizer and model
model_name = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Reading Training Data - contains 50K troll & non-troll tweets

train_data_readin = pd.read_csv('train_raw.csv')
train_data_readin.head(5)

,tweetid,user_display_name,tweet_text,clean_text,troll_or_not
0,1245883557362282497,85c9M6CDZxgBwoEye0rF12ZBgGl3xvz6Bnbvhp7MUKI=,"having each tiny wish come true, or having som...",having each tiny wish come true or having some...,1
1,961577921461866496,曲剑明,＠null It is 12:25 UTC now,null It is UTC now,1
2,941616158075211776,IFL1E0m0SRX2cdOtuLFV7xKtnBgxagKzNgkuGFvNtvs=,British number two Bedene to switch back to Sl...,British number two Bedene to switch back to Sl...,1
3,850414479976345600,Klausv,kalamitykait Thanks for bearing with us - you ...,kalamitykait Thanks for bearing with us you sh...,1
4,960784360071925760,曲剑明,＠null It is 08:56 CET now,null It is CET now,1


In [11]:
# keeping clean_text and target column, which is troll_or_not
train_data_readin = train_data_readin.loc[:,["clean_text", "troll_or_not"]]

In [12]:
train_data_readin.head(5)

,clean_text,troll_or_not
0,having each tiny wish come true or having some...,1
1,null It is UTC now,1
2,British number two Bedene to switch back to Sl...,1
3,kalamitykait Thanks for bearing with us you sh...,1
4,null It is CET now,1


In [13]:
train_data, val_data = train_test_split(train_data_readin, train_size=0.8,stratify=train_data_readin.troll_or_not, random_state=1234)

In [14]:
print("training data : ",train_data.shape)
print("validation data : ",val_data.shape)

training data :  (71958, 2)
validation data :  (17990, 2)


In [15]:
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "eval": Dataset.from_pandas(val_data)})

In [16]:
print("Dataset Dict:\n", raw_datasets)
print("\n\nTrain's features:\n", raw_datasets["train"].features)
print("\n\nFirst row of Train:\n", raw_datasets["train"][0])

Dataset Dict:
 DatasetDict({
    train: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__'],
        num_rows: 71958
    })
    eval: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__'],
        num_rows: 17990
    })
})


Train's features:
 {'clean_text': Value(dtype='string', id=None), 'troll_or_not': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None)}


First row of Train:
 {'clean_text': 'Welcome to the age of Hygiene Theater', 'troll_or_not': 0, '__index_level_0__': 45945}


In [17]:
# T5 requires a text input with a prefix
def preprocess_function(examples):
    # Add a prefix for classification task
    inputs = ["classify: " + example for example in examples["clean_text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    return model_inputs

In [18]:
# Tokenize datasets
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/71958 [00:00<?, ? examples/s]

Map:   0%|          | 0/17990 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 71958
    })
    eval: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 17990
    })
})


In [19]:
#validating for completeness
print(tokenized_datasets["train"][0])

{'clean_text': 'Welcome to the age of Hygiene Theater', 'troll_or_not': 0, '__index_level_0__': 45945, 'input_ids': [853, 4921, 10, 5242, 12, 8, 1246, 13, 5555, 122, 15067, 10639, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [20]:
tokenized_datasets = tokenized_datasets.remove_columns(["clean_text", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("troll_or_not", "labels")
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 71958
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 17990
    })
})


In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
def compute_metrics(eval_preds):
    # T5 models return a tuple where the first element contains the logits
    logits = eval_preds.predictions[0] if isinstance(eval_preds.predictions, tuple) else eval_preds.predictions
    labels = eval_preds.label_ids

    # Get the predicted class (0 or 1)
    predictions = np.argmax(logits, axis=-1)

    accuracy = evaluate.load("accuracy")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")
    f1 = evaluate.load("f1")

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions, references=labels, average='binary')["precision"],
        "recall": recall.compute(predictions=predictions, references=labels, average='binary')["recall"],
        "f1": f1.compute(predictions=predictions, references=labels, average='binary')["f1"],
    }

In [27]:
# Training arguments
training_args = TrainingArguments(
    output_dir="t5-small-troll-classifier",
    num_train_epochs=1,
    per_device_train_batch_size=8,  # T5 is larger, so smaller batch size
    per_device_eval_batch_size=8,
    weight_decay=5e-4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-27-7cbef1b4c4f5>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.163800,0.414976,0.904947,0.910908,0.897530,0.904169


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=8995, training_loss=0.1825406857501672, metrics={'train_runtime': 744.8089, 'train_samples_per_second': 96.613, 'train_steps_per_second': 12.077, 'total_flos': 644783168494176.0, 'train_loss': 0.1825406857501672, 'epoch': 1.0})

In [29]:
# Make prediction on evaluation dataset
eval_predictions = trainer.predict(tokenized_datasets["eval"])
# Handle T5's output format correctly
logits = eval_predictions.predictions[0] if isinstance(eval_predictions.predictions, tuple) else eval_predictions.predictions
y_pred = np.argmax(logits, axis=-1)

# Get the true labels
y_true = tokenized_datasets["eval"]["labels"]
y_true = np.array(y_true)

# Print the classification report
print("Validation Results:")
print(classification_report(y_true, y_pred, digits=3))

Validation Results:
              precision    recall  f1-score   support

           0      0.899     0.912     0.906      9002
           1      0.911     0.898     0.904      8988

    accuracy                          0.905     17990
   macro avg      0.905     0.905     0.905     17990
weighted avg      0.905     0.905     0.905     17990



In [30]:
trainer.save_model('/content/drive/MyDrive/TrollDetection-T5-Small')

# **Test Data Performance Checks**

In [31]:
uploaded = files.upload()

Saving validate.csv to validate.csv


In [32]:
# Reading Test Data - contains 10K troll & non-troll tweets

test_data_readin = pd.read_csv('validate.csv')
print(test_data_readin.shape)
test_data_readin.head(5)

(10000, 5)


,tweetid,user_display_name,tweet_text,clean_text,troll_or_not
0,1189300879905755142,AbacusNews,"Hungry? With a tap on your phone, you can summ...",Hungry With a tap on your phone you can summon...,0
1,1301520005822058496,guardian,US jobless claims drop sharply as government c...,US jobless claims drop sharply as government c...,0
2,359158016613949441,ZTUhI030WiV97FLuXgAU427wsJ1TtqtTBMXjKgShnU=,IT GOES SOMETHING LIKE THIS,IT GOES SOMETHING LIKE THIS,1
3,1278149046973001729,FrankLuntz,"In Arizona, coronavirus isn’t just a pandemic....",In Arizona coronavirus isnt just a pandemic It...,0
4,1301895307098230785,FoxFriendsFirst,‘100 Days Of Liberal Lawlessness’: Attorneys G...,Days Of Liberal Lawlessness Attorneys General ...,0


In [33]:
test_data_readin = test_data_readin.loc[:,["clean_text", "troll_or_not"]]

test_df = Dataset.from_pandas(test_data_readin)

# Preprocess test data
def preprocess_test(examples):
  inputs = ["classify: " + example for example in examples["clean_text"]]
  return tokenizer(inputs, max_length=512, truncation=True)

test_df = test_df.map(preprocess_test, batched=True)

# Make predictions on test data
test_predictions = trainer.predict(test_df)

test_logits = test_predictions.predictions[0] if isinstance(test_predictions.predictions, tuple) else test_predictions.predictions
test_preds = np.argmax(test_logits, axis=-1)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [34]:
# unseen data performance
print(classification_report(test_data_readin['troll_or_not'], test_preds, digits=3))

              precision    recall  f1-score   support

           0      0.901     0.916     0.908      4939
           1      0.917     0.902     0.909      5061

    accuracy                          0.909     10000
   macro avg      0.909     0.909     0.909     10000
weighted avg      0.909     0.909     0.909     10000

